In [1]:
# from datahelpers.community_tools import prepare_graphdf, prepare_graph_from_df
# from datahelpers.community_tools import meta_calculate_comms

install datahelpes from here : https://github.com/alexander-belikov/datahelpers
also pandas and igraph should be installed

In [1]:
from itertools import product
from pprint import pprint
from pandas import HDFStore, read_hdf
import pandas as pd
from os.path import expanduser

In [2]:
versions = [8, 11]
df_types = ['lit', 'gw']

fnames = ['edges_all_{0}{1}.h5'.format(t, v) for t, v in zip(df_types, versions)]
print(fnames)

['edges_all_lit8.h5', 'edges_all_gw11.h5']


## Check the keys in hdf files

In [3]:
keys = {}
index_list = []
for f in fnames:
    store = HDFStore(expanduser(f), mode='r')
    # store = HDFStore(f, mode='r')
    # store = read_hdf(f, '/y2013')
    # print(store)
    keys[f] = sorted(store.keys())
    index_list += store.keys()   

In [ ]:
# See which years are contained in both database
# index = sorted(list(set(index_list)))
# index = [year[-4:] for year in index]
# print(index)
# data = [[year] + [('/y' + year in keys[f]) for f in keys.keys()] for year in index]
# print(data)
# df = pd.DataFrame(data)
# df.set_index(0, inplace=True)
# df.columns = ['literome', 'geneways']

## practice merging:

In [6]:
df_1 = read_hdf('edges_all_lit8.h5', '/y2013')
df_2 = read_hdf('edges_all_gw11.h5', '/y2013')
edges= {}
for i in df_1.index:
    key = str(df_1['up'][i]) + '_' + str(df_1['dn'][i])
    value = df_1[0][i]
    edges[key] = value

for i in df_2.index:
    key = str(df_2['up'][i]) + '_' + str(df_1['dn'][i])
    value = df_1[0][i]
    if key in edges.keys():
        edges[key] = max(edges[key], value)
    else:
        edges[key] = value
print(len(edges))

165143


Please be advised that there are genes are both up and dn.

In [8]:
undirected_edges = {}
coupled_1 = 0
coupled = 0
uncoupled = 0
same = 0
for edge in edges.keys():
    up, down = edge.split('_')
    if up == down:
        print('{}: {}'.format(same, up))
        same += 1
    reversed_edge = down + '_' + up
    if reversed_edge in edges:
        coupled_1 += 1
        if reversed_edge not in undirected_edges:
            undirected_edges[edge] = max(edges[edge], edges[reversed_edge])
            coupled += 1
    else:
        undirected_edges[edge] = edges[edge]
        uncoupled += 1
print(len(undirected_edges))
print('coupled = {}, uncoupled = {}'.format(coupled, uncoupled))
print(coupled * 2 + uncoupled)
print(coupled_1)

0: 551
1: 1950
2: 1958
3: 3458
4: 3480
5: 3553
6: 3565
7: 3630
8: 5347
9: 5617
10: 5921
11: 6446
12: 7001
13: 7124
14: 7157
15: 22974
136790
coupled = 28369, uncoupled = 108421
165159
56722


In [ ]:
def merge(fnames, year):
    df_dict = {f: read_hdf(f, '/y' + year) for f in fnames}
    

## prepare_graphdf fetches the edges from a list hdf files that are under the key, e.g. 1995

In [6]:
df = prepare_graphdf(fnames, 'edges', 1995)

NameError: name 'prepare_graphdf' is not defined

In [6]:
df.shape

(28644, 3)

In [7]:
df.head()

,up,dn,0
0,2,213,2
1,2,348,2
2,2,462,4
3,2,718,1
4,2,973,3


## prepare_graph_from_df creates a graph (that has integer nodes) and provides weights and also a map i -> actual node

In [8]:
g, weights, inv_conv = prepare_graph_from_df(df, 'edges', directed=False, percentile_value=None)